## Struktury danych

In [ ]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y
        
class Segment:
    def __init__(self,p,q):
        if p.x<q.x:
            left,right=p,q
        else:
            left,right=q,p
        
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) # zakladam ze nie ma pionowych odcinkow
        self.b=left.y-left.x*self.a
        

class Trapezoid:

    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None
